In [1]:
from pathlib import Path
from yaml import load as loady, dump as dumpy, Loader, Dumper
new_system = 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
uscore_base = 'http://hl7.org/fhir/us/core'
uscore_types = {
'AllergyIntolerance': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance|6.1.0',
'CarePlan': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan|6.1.0',
'CareTeam': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam|6.1.0',
'Condition': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-problems-health-concerns|6.1.0',
'Coverage': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-coverage|6.1.0',
'Device': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device|6.1.0',
'DiagnosticReport': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab|6.1.0',
'DocumentReference': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference|6.1.0',
'Encounter': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter|6.1.0',
'Goal': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal|6.1.0',
'Immunization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization|6.1.0',
'Location': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location|6.1.0',
'Medication': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication|6.1.0',
'MedicationRequest': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest|6.1.0',
'MedicationDispense': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationdispense|6.1.0',
'Observation': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab|6.1.0',
'Organization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization|6.1.0',
'Patient': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient|6.1.0',
'Practitioner': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner|6.1.0',
'PractitionerRole': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole|6.1.0',
'Procedure': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure|6.1.0',
'Provenance': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-provenance|6.1.0',
'RelatedPerson': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-relatedperson|6.1.0',
'ServiceRequest': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-servicerequest|6.1.0',
'Specimen': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-specimen|6.1.0',
}


In [2]:
from nested_lookup import nested_lookup
# print(nested_lookup('severity', d , with_keys=True))

def get_paths(d, my_key, value, current = ''):
#   print(f'value = {value}')
  for a, b in d.items():
    # print(f'Key = {a}, Value = {b} b.type = {type(b)} b == value ?: {b == value}')

    # print(f'Current = {current}')
    if a==my_key and b == value:
      current = current+f"['{a}']"
      print(f'Found {value} at {current}')
      yield current
    elif isinstance(b, dict):
    #   print(f'Key = {a}, Dict Value = {b}')
      yield from get_paths(b, my_key, value, current+f"['{a}']",)
    elif isinstance(b, list):
      if value in b:
        yield current
      for i,j in enumerate(b):
        # print(f'List item {i} = {j}')
        try:
          yield from get_paths(j, my_key, value, current+f"['{a}'][{i}]",)
        except AttributeError:
            pass



in_path = Path(r'input/examples-yaml')
print(in_path)
for counter,i in enumerate(in_path.glob('*.yml')):
    print(f'=== {counter}: {i} ===')
    obj = loady(i.read_text(), Loader=Loader)
    # print(obj['id'])
    example_types = set(nested_lookup('resourceType', obj))
    if example_types & set(uscore_types): # only process if US Core type found
      for example_type in example_types:
          if example_type in uscore_types:
            print(f'US Core type: {example_type}')
            paths = list(get_paths(obj,'resourceType', example_type))
            print(paths)
            print()
            for path in paths:
              
              print(f'Path = {path}')
              exec(f"print(obj{path})")
              if 'contained' in path:
                add_meta = input(f'Add meta for {example_type} profile? Type "Y" or "N"')
                if add_meta != 'Y':
                   continue
                else:
                  print('=========== Contained resource ===========')
                  break

              try:
                exec(f"""print(obj{path.replace("['resourceType']","['meta']['profile']")})""")
              except KeyError:
                print('====== No meta profile ======')
              new_url = uscore_types[example_type]
              print(f'new_url ={new_url}')

              try:
                  exec(f"""obj{path.replace("['resourceType']","['meta']['profile']")}={[new_url]}""")
              except KeyError:
                  exec(f"""obj{path.replace("['resourceType']","['meta']")} = {{'profile': [new_url]}}""")

              print(dumpy(obj, Dumper=Dumper,sort_keys=False))



      i.write_text(dumpy(obj, Dumper=Dumper,sort_keys=False)) # update only if US Core type found










input/examples-yaml
=== 0: input/examples-yaml/cdex-questionnaireresponse-example1.yml ===
=== 1: input/examples-yaml/cdex-questionnaireresponse-example3.yml ===
=== 2: input/examples-yaml/cdex-parameters-example4.yml ===
US Core type: Observation
Found Observation at ['parameter'][7]['part'][2]['resource']['resourceType']
Found Observation at ['parameter'][8]['part'][2]['resource']['resourceType']
Found Observation at ['parameter'][9]['part'][2]['resource']['resourceType']
["['parameter'][7]['part'][2]['resource']['resourceType']", "['parameter'][8]['part'][2]['resource']['resourceType']", "['parameter'][9]['part'][2]['resource']['resourceType']"]

Path = ['parameter'][7]['part'][2]['resource']['resourceType']
Observation
['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab|6.1.0']
new_url =http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab|6.1.0
resourceType: Parameters
id: cdex-parameters-example4
meta:
  extension:
  - url: http://hl7.org/f